# L7 Construire un système de bout en bout

## Configuration
#### Chargement de la clé OpenAI et des librairies Python

In [3]:
import os
from openai import OpenAI
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)

In [2]:
def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

## Système de prompts en chaîne pour traiter la demande de l'utilisateur

In [4]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Étape 1 : Vérifiez l'entrée pour voir si elle signale l'API de modération ou s'il s'agit d'une injection de prompt.
    response = client.moderations.create(input=user_input)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        print("Étape 1 : Entrée signalée par l'API de modération. ")
        return "Désolé, nous ne pouvons traiter votre requête."

    if debug: print("Étape 1 : L'entrée a passé la vérification de modération.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    category_and_product_response = '\n'.join([line for line in category_and_product_response.split('\n') if not line.strip().startswith('```')])
    # Étape 2: Extraire la liste des produits
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("Étape 2 : Extraire la liste des produits")

    # Étape 3 : Si des produits sont trouvés on les récupère
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Étape 3 : Informations sur le produit recherchées.")

    # Étape 4: Répondre à la question de l'utilisateur
    system_message = f"""Vous êtes un assistant du service client pour un 
    grand magasin d'électronique. Répondez avec un ton amical et serviable, 
    en fournissant des réponses concises. Assurez-vous de poser à l'utilisateur 
    des questions de suivi pertinentes.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Informations intéressantes pour le produit :\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Étape 4 : Réponse générée à la question de l'utilisateur.")
    all_messages = all_messages + messages[1:]

    # Étape 5: Soumettez la réponse à l'API de modération.
    response = client.moderations.create(input=final_response)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        if debug: print("Étape 5 : Réponse signalée par l'API de modération.")
        return "Désolé, nous ne pouvons pas fournir cette information."

    if debug: print("Étape 5 : Réponse ayant passé la vérification de modération.")

    # Étape 6 : Demandez au modèle si la réponse répond bien à la demande initiale de l'utilisateur.
    user_message = f"""
    Message de l'utilisateur : {delimiter}{user_input}{delimiter}
    Réponse de l'agent : {delimiter}{final_response}{delimiter}

    La réponse répond-elle suffisamment à la question ?

    Répondez par "Y" ou "N".
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Étape 6 : Le modèle a évalué la réponse.")

    # Étape 7 : Si oui, utilisez cette réponse ; sinon, dites que vous allez mettre l'utilisateur en relation avec un humain. 
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Étape 7 : Le modèle a approuvé la réponse.")
        return final_response, all_messages
    else:
        if debug: print("Étape 7 : Le modèle a désapprouvé la réponse.")
        neg_str = "Je ne peux pas fournir les informations que vous recherchez. Je vais vous mettre en relation avec un représentant humain pour une assistance supplémentaire. "
        return neg_str, all_messages

user_input = "Parlez-moi du SmartX ProPhone et de l'appareil photo FotoSnap, le reflex numérique. Parlez-moi également de vos téléviseurs."
response,_ = process_user_message(user_input,[])
print(response)

Étape 1 : L'entrée a passé la vérification de modération.
Error: Invalid JSON string
Étape 2: Extraire la liste des produits
Étape 3 : Informations sur le produit recherchées.
Étape 4 : Réponse générée à la question de l'utilisateur.
Étape 5 : Réponse ayant passé la vérification de modération.
Étape 6 : Le modèle a évalué la réponse.
Étape 7 : Le modèle a désapprouvé la réponse.
Je ne peux pas fournir les informations que vous recherchez. Je vais vous mettre en relation avec un représentant humain pour une assistance supplémentaire. 


### Fonction qui collecte les messages de l'utilisateur et de l'assistant au fil du temps

In [ ]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Discutez avec le chatbot !
Notez que le message système inclut des instructions détaillées sur ce que l'OrderBot doit faire.

In [ ]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"Vous êtes un assistant de service."} ]  

inp = pn.widgets.TextInput( placeholder='Entrez votre texte ici …')
button_conversation = pn.widgets.Button(name="Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard